In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import argparse
from inferencers import *
import logging
from transformers import AutoTokenizer, CLIPModel,AutoProcessor
import sys
sys.path.append('/data/chy/online')
from open_flamingo_v2.open_flamingo.src.factory import create_model_and_transforms
from transformers import IdeficsForVisionText2Text, AutoProcessor
import json
from PIL import Image
import requests

In [ ]:
device_set = "cuda:0"
device = torch.device(device_set)

In [ ]:
model, image_processor, tokenizer = create_model_and_transforms(
            clip_vision_encoder_path="ViT-L-14",
            clip_vision_encoder_pretrained="openai",
            lang_encoder_path="/data/share/mpt-7b/",
            tokenizer_path="/data/share/mpt-7b/",
            cross_attn_every_n_layers=4,
            precision="fp16",
            inference=True,
            device=device_set,
            checkpoint_path="/data/share/OpenFlamingo-9B-vitl-mpt7b/checkpoint.pt"
        )

In [ ]:
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
preprocessed = image_processor(images=demo_image_one,return_tensors="pt")

In [ ]:
print("preprocessed:",preprocessed)

In [ ]:
print("preprocessed shape",preprocessed.shape)

In [ ]:
# 直接用 OpenFlamingo 抽取特征
with torch.no_grad():
    vision_x = model.vision_encoder(preprocessed.pixel_values)[1]  # 获取编码器输出，Shape: (1, 256, 1024)

# 检查特征维度
print("Image features shape:", vision_x.shape)  # 输出应为 (1, 256, 1024) - 一张图片的 (256, 1024) 特征

In [1]:
import torch
from transformers import  AutoProcessor,IdeficsForVisionText2Text
from PIL import Image
import requests


device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "/data1/pyz/model_weight/idefics-9b"
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [52]:
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
preprocessed = processor.image_processor(demo_image_one,return_tensors="pt")

print("preprocessed shape",preprocessed.shape) # preprocessed shape torch.Size([1, 3, 224, 224])

preprocessed shape torch.Size([1, 3, 224, 224])


AttributeError: 'IdeficsForVisionText2Text' object has no attribute 'vision_encoder'

In [19]:
prompt = ["User:",
          demo_image_one,
           "Classify the following image into a single category.\nAssistant: vegetables.\n",
           "User:",
          demo_image_one,
           "Classify the following image into a single category.\nAssistant: vegetables.\n",
           "User:",
           demo_image_one,
           "Classify the following image into a single category.\nAssistant:"
           ]

In [33]:
image_list = [demo_image_one,demo_image_one,demo_image_one]

In [34]:
inputs = processor(prompt, return_tensors="pt").to("cuda")

In [40]:
preprocessed = processor.image_processor(image_list)
print("preprocessed shape",preprocessed.shape) # preprocessed shape torch.Size([3, 3, 224, 224])

preprocessed shape torch.Size([3, 3, 224, 224])


In [41]:
print(inputs["pixel_values"].shape)

torch.Size([1, 3, 3, 224, 224])


In [36]:
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

In [37]:
output = model.generate(
    **inputs,
    max_new_tokens=20,
    num_beams=1,
    output_scores=True,
    return_dict_in_generate=True
)

In [38]:
print(output)

GreedySearchDecoderOnlyOutput(sequences=tensor([[    1,  4911, 29901, 32000, 32001, 32000,  4134,  1598,   278,  1494,
          1967,   964,   263,  2323,  7663, 29889,    13,  7900, 22137, 29901,
         18655,  1849, 29889,    13,  2659, 29901, 32000, 32001, 32000,  4134,
          1598,   278,  1494,  1967,   964,   263,  2323,  7663, 29889,    13,
          7900, 22137, 29901, 18655,  1849, 29889,    13,  2659, 29901, 32000,
         32001, 32000,  4134,  1598,   278,  1494,  1967,   964,   263,  2323,
          7663, 29889,    13,  7900, 22137, 29901, 18655,  1849, 29889,    13,
          2659, 29901, 32000, 32001, 32000,  4134,  1598,   278,  1494,  1967,
           964,   263,  2323,  7663, 29889,    13]], device='cuda:0'), scores=(tensor([[-11.8125, -16.5000,  -3.5625,  ...,  -9.6250,  -1.1016,  -6.0312]],
       device='cuda:0', dtype=torch.bfloat16), tensor([[ -9.2500, -13.6250,  -5.4688,  ...,  -8.1875,  -5.1875,  -6.2500]],
       device='cuda:0', dtype=torch.bfloat16), t

In [39]:
# 获取模型的所有方法和属性
print(dir(model)) # 'get_input_embeddings', 'get_memory_footprint', 'get_output_embeddings'

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_auto_class', '_backward_compatibility_gradient_checkpointing', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_convert_head_mask_to_5d', '_create_repo', '_dispatch_accelerate_model', '_expand_inputs_for_generation', '_extract_past_from_model_output', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_from_config', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_decoder_start_token_id', '_get_files_timestamps', '_get_generation_mode', '_get_logits_processor', '_get_logits_war

In [68]:
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
image = [demo_image_one]

preprocessed = processor(image, return_tensors="pt").pixel_values

print("preprocessed shape",preprocessed.shape) # preprocessed shape torch.Size([1, 3, 224, 224])

preprocessed shape torch.Size([1, 1, 3, 224, 224])


In [69]:
preprocessed = preprocessed.squeeze(0)
print("preprocessed shape",preprocessed.shape) 

preprocessed shape torch.Size([1, 3, 224, 224])


In [72]:
model.model.get_input_embeddings()

IdeficsDecoupledEmbedding(
  num_embeddings=32000, num_additional_embeddings=2, embedding_dim=4096, partially_freeze=False
  (additional_embedding): Embedding(2, 4096)
)

In [70]:
model.model.vision_model(preprocessed.to(device))

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0398,  0.5781,  0.4551,  ...,  0.0835,  0.1226,  0.7891],
         [ 0.8828, -0.5586,  0.7578,  ..., -0.1934, -1.0078,  0.1621],
         [ 1.5625, -0.2129,  0.6133,  ..., -0.1641, -0.9805,  0.3340],
         ...,
         [ 0.1660, -0.4004,  0.1973,  ...,  0.2305,  0.3164, -0.1504],
         [ 0.1465, -0.3438,  0.1289,  ...,  0.3555,  0.5469, -0.1543],
         [-0.0908, -0.1465,  0.2852,  ...,  0.1562,  0.6484, -0.3555]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<AddBackward0>), pooler_output=tensor([[-0.3574,  0.6016,  0.7070,  ..., -0.0332, -0.0144,  1.4219]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [79]:
print(model.model.vision_model(preprocessed.to(device)).last_hidden_state.shape)

torch.Size([1, 257, 1280])


In [78]:
last_hidden_state = model.model.vision_model(preprocessed.to(device)).last_hidden_state

In [80]:
print(last_hidden_state)

tensor([[[ 0.0398,  0.5781,  0.4551,  ...,  0.0835,  0.1226,  0.7891],
         [ 0.8828, -0.5586,  0.7578,  ..., -0.1934, -1.0078,  0.1621],
         [ 1.5625, -0.2129,  0.6133,  ..., -0.1641, -0.9805,  0.3340],
         ...,
         [ 0.1660, -0.4004,  0.1973,  ...,  0.2305,  0.3164, -0.1504],
         [ 0.1465, -0.3438,  0.1289,  ...,  0.3555,  0.5469, -0.1543],
         [-0.0908, -0.1465,  0.2852,  ...,  0.1562,  0.6484, -0.3555]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<AddBackward0>)
